<a href="https://colab.research.google.com/github/yukinaga/bayesian_statistics/blob/main/section_4/01_bayesian_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ベイズ推定の実装
ベイズ推定をPythonのコードで実装しましょう。  
例として、あるウェブサイトにおけるボタンのクリック率を、分布として推定します。  

## ベイズの定理と確率分布
ベイズの定理を確率分布に適用します。  

$$p(\theta\mid r) = \frac{p(r\mid\theta)p(\theta)}{p(r)}$$

この式において、$\theta$は確率分布のパラメータ、$r$は得られたデータです。  
$p(\theta)$は事前分布、$p(\theta\mid r)$が事後分布、$p(r\mid\theta)$は尤度（尤度関数）と呼ばれます。  
$p(r)$は「正規化定数」などと呼ばれますが、$\theta$を含まない定数となります。  
$p(r)$は事後分布$p(\theta\mid r)$の値を0から1の範囲に納め、合計1にする、いわゆる「正規化」の役割を担います。  

## ベルヌーイ分布

あるウェブサイトにボタンを配置します。  
このボタンがクリックされる確率を$\theta$、クリックされることを1、クリックされないことを0とすると、条件付き確率を以下のように表すことができます。  

$$p(r=1\mid\theta)=\theta$$
$$p(r=0\mid\theta)=1-\theta$$

これらの式は、以下の表にまとめることができます。  

$$p(r\mid\theta)=\theta^{r}(1-\theta)^{1-r}$$

この確率分布は、「ベルヌーイ分布」（Bernoulli distribution）と呼ばれます。  
ベルヌーイ分布におけるパラメータ$\theta$は、クリックされる確率なので0から1までの範囲をとります。  
今回は、このベルヌーイ分布を尤度として使います。  

## 事前分布の用意
今回は、ベイズ推定により、事後分布$p(\theta\mid r)$を推定します。  
まずは、事前分布$p(\theta)$を用意します。  
今回は、全く手がかりが無いので確率変数によらず確率の値が一定な、一様分布を事前分布に使います。  

In [ ]:
import numpy as np

thetas = np.linspace(0, 1, 500)  # パラメータ: 0から1の範囲
p = np.ones(len(thetas)) / len(thetas)  # 事前確率の分布: 一様分布
print(p)

## 尤度を計算する関数
ベイズの定理における尤度$p(r\mid\theta)$を計算するための関数を用意します。  
今回は試行が一回のみのため、尤度はベルヌーイ分布で表すことができます。  

In [ ]:
def likehood(r):
    # ベルヌーイ分布
    if r==1:  # クリックされた場合
        return thetas
    else:  # クリックされなかった場合
        return 1-thetas

## 事後分布を計算する関数
事後分布$p(\theta\mid r)$を計算するための関数を設定します。  
事前確率と尤度の積をとり、正規化定数$p(r)$で割ります。  
正規化定数は積分により数式の形で表せる場合もあるのですが、一般的に数式の形を求めるのは難しいです。    
従って、今回は事前確率と尤度の積を、その総和で割ることで正規化することにします。  

In [ ]:
def posterior(r, prior):
    lipr = prior * likehood(r)  # 事前確率と尤度の積
    return lipr / lipr.sum()  # 正規化: 事後分布の値を0から1の範囲に納め、合計1に

## 事後分布の計算
まずは、1回ボタンを表示して、クリックされた場合の事後分布を計算します。  
先ほど設定した`posterior()`関数に、クリックされたことを表す1と事前分布を渡します。  

In [ ]:
p = posterior(1, p)  # 事後分布の計算: クリックした場合
print(p)

## 事後分布の可視化
Matplotlibを使い、事後分布を可視化します。  

In [ ]:
import matplotlib.pyplot as plt

plt.style.use("seaborn-darkgrid")

plt.plot(thetas, p)
plt.xlabel(r"$\theta$")
plt.ylabel(r"$p(\theta)$")
plt.show()

まだ一回しかボタンが表示されていないので、分布は直線上になります。

## ベイズ更新
1回の結果が得られる度に、事後分布を更新します。  
ボタンが合計20回表示され、2回クリックされた後の事後分布を計算し視覚化します。

In [ ]:
n = 20  # ボタンの表示数
clicks = 2  # クリック数

p = np.ones(len(thetas)) / len(thetas)  # 事前分布

# クリックした場合
for i in range(clicks):
    p = posterior(1, p)  # 1: クリックした

# クリックしなかった場合
for i in range(n-clicks):
    p = posterior(0, p)  # 0: クリックしなかった

plt.plot(thetas, p)
plt.xlabel(r"$\theta$")
plt.ylabel(r"$p(\theta)$")
plt.show()

確率分布にピークが出現しました。  
データのクリック率は10％ですが、その付近が頂点となっています。  

試行を重ねると、どのように分布が変化するのか確認しましょう。  

In [ ]:
def show_posterior(clicks, n):
    p = np.ones(len(thetas)) / len(thetas)  # 事前分布
    for i in range(clicks):
        p = posterior(1, p)  # 1: クリックした
    for i in range(n-clicks):
        p = posterior(0, p)  # 0: クリックしなかった
    plt.plot(thetas, p, label=str(clicks)+"/"+str(n))

show_posterior(1, 5)
show_posterior(2, 20)
show_posterior(5, 100)

plt.xlim(0, 0.5)
plt.xlabel(r"$\theta$")
plt.ylabel(r"$p(\theta)$")
plt.legend()
plt.show()

試行回数が増えると、次第に分布の裾野が狭くなり推定の精度が向上することを確認できます。